In [1]:
import wandb
wandb.login(key = "")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
from transformers import MarianTokenizer
from datasets import load_dataset
from transformers import MarianMTModel
from datasets import load_from_disk
# Prepare training arguments
from transformers import TrainingArguments, Trainer

In [3]:
# Load the dataset
dataset = load_dataset("opus_books", "en-es")


print(dataset)
print(dataset["train"][0])


Generating train split:   0%|          | 0/93470 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 93470
    })
})
{'id': '0', 'translation': {'en': 'Source: Project GutenbergAudiobook available here', 'es': 'Source: Wikisource & librodot.com'}}


In [9]:
# tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-es')
tokenizer = MarianTokenizer.from_pretrained('/kaggle/working/path_to_save/tokenizer')

# Tokenize the dataset

def tokenize_function(examples):
    # Extract English ('en') and Spanish ('es') texts
    en_texts = [example["en"] for example in examples["translation"]]
    es_texts = [example["es"] for example in examples["translation"]]
    
    # Tokenize the English texts (inputs) and Spanish texts (labels)
    inputs = tokenizer(en_texts, padding="max_length", truncation=True)
    labels = tokenizer(es_texts, padding="max_length", truncation=True)
    
    # Return tokenized inputs and corresponding labels
    inputs["labels"] = labels["input_ids"]
    #print(inputs)
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/93470 [00:00<?, ? examples/s]

In [10]:

#print(tokenized_datasets["train"][0])

split_tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.15)

# Access the tokenized train and test sets
tokenized_train = split_tokenized_datasets["train"]
tokenized_test = split_tokenized_datasets["test"]

# Check the number of samples in each set
print(f"Training set: {len(tokenized_train)} samples")
print(f"Test set: {len(tokenized_test)} samples")



Training set: 79449 samples
Test set: 14021 samples


In [7]:
tokenizer.save_pretrained("path_to_save/tokenizer")

('path_to_save/tokenizer/tokenizer_config.json',
 'path_to_save/tokenizer/special_tokens_map.json',
 'path_to_save/tokenizer/vocab.json',
 'path_to_save/tokenizer/source.spm',
 'path_to_save/tokenizer/target.spm',
 'path_to_save/tokenizer/added_tokens.json')

In [ ]:

# tokenized_train.save_to_disk("path_to_save/tokenized_train")
# tokenized_test.save_to_disk("path_to_save/tokenized_test")

In [24]:
tokenize_train = load_from_disk("/kaggle/input/train-nlp")
tokenize_test = load_from_disk("/kaggle/input/test-nlp")


In [12]:
# tokenize_test

In [9]:
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-es')



pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
from transformers import TrainerCallback

class CustomCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        print(f"Saving model at step {state.global_step}")

    def on_evaluate(self, args, state, control, **kwargs):
        print(f"Evaluating model at step {state.global_step}")


In [11]:


training_args = TrainingArguments(
    output_dir="./results",          # output directory
    evaluation_strategy="epoch",     # evaluate every epoch
    save_strategy="steps",           # save checkpoints every specified number of steps
    save_steps=500,                  # save every 500 steps
    per_device_train_batch_size=8,   # batch size
    per_device_eval_batch_size=8,    # batch size for evaluation
    num_train_epochs=3,              # number of epochs
    save_total_limit=3,              # limit the total number of checkpoints
)

# Prepare Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenize_train,
    eval_dataset=tokenize_test,
    callbacks=[CustomCallback]
)



/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
import torch

torch.cuda.empty_cache()



In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: myselfmichael14 (myselfmichael14-na). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.166200,0.150085
2,0.143000,0.139711
3,0.131600,0.136862


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


Saving model at step 500


...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


Saving model at step 14898


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluating model at step 14898


TrainOutput(global_step=14898, training_loss=0.15597833686426096, metrics={'train_runtime': 15377.7168, 'train_samples_per_second': 15.5, 'train_steps_per_second': 0.969, 'total_flos': 3.2318294341976064e+16, 'train_loss': 0.15597833686426096, 'epoch': 3.0})

In [14]:
trainer.evaluate()


Evaluating model at step 14898


{'eval_loss': 0.13686226308345795,
 'eval_runtime': 364.7552,
 'eval_samples_per_second': 38.439,
 'eval_steps_per_second': 2.404,
 'epoch': 3.0}

In [15]:
model.save_pretrained("path_to_save/model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


In [13]:
model = MarianMTModel.from_pretrained('/kaggle/working/path_to_save/model')


In [17]:

# Check if a GPU (CUDA) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device (GPU or CPU)
model.to(device)

# Sample input text (English sentence for translation)
input_text = "Mery Christmas"

# Tokenize the input text and move the tensors to the same device
encoded_input = tokenizer(input_text, return_tensors='pt').to(device)

# Generate translation output from the model (also on the same device)
translation_tokens = model.generate(**encoded_input)

# Decode the output token IDs into a readable string
translated_text = tokenizer.decode(translation_tokens[0], skip_special_tokens=True)

# Display the translated text
print(f"Translated text: {translated_text}")

Translated text: ¡Merezca la Navidad!


In [22]:
inputs = [example['translation']['en'] for example in tokenize_test]  # assuming English to Spanish translation
references = [[example['translation']['es']] for example in tokenize_test]  # references must be a list of lists

In [23]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tokenizer(batch, return_tensors="pt", padding=True, truncation=True)

# Create a DataLoader for batch processing
loader = DataLoader(inputs, batch_size=32, collate_fn=collate_fn)

# Translate in batches
translated_texts = []
model.eval()  # set model to evaluation mode
with torch.no_grad():
    for batch in loader:
        inputs_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        outputs = model.generate(input_ids=inputs_ids, attention_mask=attention_mask)
        translations = [tokenizer.decode(t, skip_special_tokens=True) for t in outputs]
        translated_texts.extend(translations)


KeyboardInterrupt: 

In [19]:
!pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f3db560dda54aac76426cc7c514589c0cb9d0bb117d5d485c4d73d476cb43647
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [20]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import torch

In [28]:
import torch
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Check if CUDA (GPU) is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def calculate_perplexity(model, tokenizer, text):
    # Tokenize input text and move it to the correct device
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(device)
    
    # Ensure model is on the same device
    model = model.to(device)
    
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
    
    # Calculate perplexity from the loss
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

def calculate_bleu_score(reference_texts, generated_text):
    # Split the reference and generated text into tokens
    references = [ref.split() for ref in reference_texts]
    candidate = generated_text.split()
    
    # Calculate BLEU score
    score = sentence_bleu(references, candidate)
    return score

def calculate_rouge_score(reference_text, generated_text):
    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # Calculate ROUGE scores
    scores = scorer.score(reference_text, generated_text)
    return scores

def calculate_f1_score(rouge_scores):
    f1_scores = {}
    # Calculate F1 score for each ROUGE metric
    for key, score in rouge_scores.items():
        precision = score.precision
        recall = score.recall
        if precision + recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0.0
        f1_scores[key + '_f1'] = f1
    return f1_scores

def evaluate_model(texts, model, tokenizer):
    results = []
    
    # Move the model to the appropriate device
    model = model.to(device)
    
    for text in texts:
        # Tokenize input text and move tensors to the correct device
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(device)
        
        # Generate text using the model
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=512,  
                num_beams=5,    
                early_stopping=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # Decode the generated output into text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Calculate metrics
        perplexity = calculate_perplexity(model, tokenizer, text)
        bleu_score = calculate_bleu_score([text], generated_text)
        rouge_scores = calculate_rouge_score(text, generated_text)
        f1_scores = calculate_f1_score(rouge_scores)
        
        # Append results
        results.append({
            'generated_text': generated_text,
            'perplexity': perplexity,
            'bleu_score': bleu_score,
            'rouge_scores': rouge_scores,
            'f1_scores': f1_scores
        })
    
    return results


In [25]:
tokenize_test

Dataset({
    features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14021
})

In [29]:
import torch

# Check if CUDA (GPU) is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def evaluate_model_on_dataset(dataset, model, tokenizer):
    results = []
    
    # Move model to the same device
    model = model.to(device)
    
    for example in dataset:
        # Decode input_ids and labels (assuming 'labels' are reference translations)
        input_text = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
        reference_text = tokenizer.decode(example['labels'], skip_special_tokens=True)
        
        # Tokenize input text
        inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512).to(device)

        # Evaluate the model on the input text
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=512,  
                num_beams=5,    
                early_stopping=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Calculate metrics
        perplexity = calculate_perplexity(model, tokenizer, input_text)
        bleu_score = calculate_bleu_score([reference_text], generated_text)
        rouge_scores = calculate_rouge_score(reference_text, generated_text)
        f1_scores = calculate_f1_score(rouge_scores)
        
        # Add the results for each example
        results.append({
            'id': example['id'],
            'input_text': input_text,
            'reference_text': reference_text,
            'generated_text': generated_text,
            'perplexity': perplexity,
            'bleu_score': bleu_score,
            'rouge_scores': rouge_scores,
            'f1_scores': f1_scores
        })
    
    return results

# Example usage
results = evaluate_model_on_dataset(tokenize_test, model, tokenizer)


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
